In [1]:
import pandas as pd
import numpy as np

### Read in data and drop rows with NA in important columns

In [20]:
df_summs_raw = pd.read_csv("all_games.csv")
df_sales_raw = pd.read_csv("vgsales.csv")
df_summs = df_summs_raw.copy()
df_sales = df_sales_raw.copy()

df_sales = df_sales.drop(["Rank"],axis=1)
df_summs = df_summs.rename(columns={'name': 'Name'})
df_summs = df_summs.rename(columns={'platform': 'Platform'})

In [21]:
# drop rows with nans in the important columns
print(df_sales.shape)
print(df_summs.shape)

df_sales = df_sales.dropna(axis=0,subset=['Name', 'Year', 'Global_Sales'])
df_summs = df_summs.dropna(axis=0,subset=['Name', 'release_date', 'summary'])

print(df_sales.shape)
print(df_summs.shape)

(16598, 10)
(18800, 6)
(16327, 10)
(18686, 6)


### Make the dfs compatible for merging

In [22]:
# Make platform names same in each dataframe
print(df_sales["Platform"].unique())
print(df_summs["Platform"].unique())

plats = {"N64":"Nintendo 64", "PS":"PlayStation", "PS3":"PlayStation 3", "DC":"Dreamcast", \
        "X360":"Xbox 360", "XOne":"Xbox One", "PS2":"PlayStation 2", "PS4":"PlayStation 4", \
        "GC":"GameCube", "PSV":"PlayStation Vita", "GBA":"Game Boy Advance", "WiiU":"Wii U", "XB":"Xbox"}
for key in plats.keys():
    df_sales['Platform'] = df_sales['Platform'].replace(key,plats[key])

['Wii' 'NES' 'GB' 'DS' 'X360' 'PS3' 'PS2' 'SNES' 'GBA' '3DS' 'PS4' 'N64'
 'PS' 'XB' 'PC' '2600' 'PSP' 'XOne' 'GC' 'WiiU' 'GEN' 'DC' 'PSV' 'SAT'
 'SCD' 'WS' 'NG' 'TG16' '3DO' 'GG' 'PCFX']
[' Nintendo 64' ' PlayStation' ' PlayStation 3' ' Dreamcast' ' Xbox 360'
 ' Wii' ' Xbox One' ' PC' ' Switch' ' PlayStation 2' ' PlayStation 4'
 ' GameCube' ' Xbox' ' Wii U' ' Game Boy Advance' ' 3DS' ' Xbox Series X'
 ' DS' ' PlayStation Vita' ' PlayStation 5' ' PSP' ' Stadia']


In [23]:
# strip any leading or trailing spaces in the columns I'm going to merge on
df_sales["Name"] = df_sales["Name"].str.strip()
df_sales["Platform"] = df_sales["Platform"].str.strip()

df_summs["Name"] = df_summs["Name"].str.strip()
df_summs["Platform"] = df_summs["Platform"].str.strip()

### Merge the dfs

In [24]:
# merge the dataframes on game name, year, and platform
df_sales["Year"] = df_sales["Year"].astype(int)
df_summs["Year"] = df_summs["release_date"].str[-4:].astype(int)
df = df_summs.merge(df_sales, how="inner", on=["Name","Year","Platform"])

In [25]:
print(df.shape)
df

(5294, 14)


,Name,Platform,release_date,summary,meta_score,user_review,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,"November 23, 1998","As a young boy, Link is tricked by Ganondorf, ...",99,9.1,1998,Action,Nintendo,4.10,1.89,1.45,0.16,7.60
1,Tony Hawk's Pro Skater 2,PlayStation,"September 20, 2000",As most major publishers' development efforts ...,98,7.4,2000,Sports,Activision,3.05,1.41,0.02,0.20,4.68
2,Grand Theft Auto IV,PlayStation 3,"April 29, 2008",[Metacritic's 2008 PS3 Game of the Year; Also ...,98,7.7,2008,Action,Take-Two Interactive,4.76,3.76,0.44,1.62,10.57
3,SoulCalibur,Dreamcast,"September 8, 1999","This is a tale of souls and swords, transcendi...",98,8.4,1999,Fighting,Namco Bandai Games,0.00,0.00,0.34,0.00,0.34
4,Grand Theft Auto IV,Xbox 360,"April 29, 2008",[Metacritic's 2008 Xbox 360 Game of the Year; ...,98,7.9,2008,Action,Take-Two Interactive,6.76,3.10,0.14,1.03,11.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5289,Pulse Racer,Xbox,"January 2, 2003",Pulse Racer takes you to a future where racers...,24,2.2,2003,Racing,Jaleco,0.01,0.00,0.00,0.00,0.02
5290,Fighter Within,Xbox One,"November 19, 2013",Unleash your inner fighter to beat your friend...,23,2.8,2013,Fighting,Ubisoft,0.11,0.07,0.00,0.02,0.19
5291,Homie Rollerz,DS,"March 5, 2008","Homie Rollerz is a fast-paced, mayhem-laden ka...",23,3.0,2008,Racing,Destineer,0.07,0.00,0.00,0.01,0.07
5292,Charlie's Angels,GameCube,"July 9, 2003","Join Natalie, Dylan, and Alex for an intense a...",23,4.3,2003,Action,Ubisoft,0.01,0.00,0.00,0.00,0.02


### Fix data types

In [26]:
# convert release_date to time
df["release_date"] = pd.to_datetime(df["release_date"])

### Remove outlier games
From EDA, I found there are outliers (all 6 of the most extreme are Wii and Mario). Words in these game descriptions will weigh too heavily and be assumed to be great words, when in reality, the descriptions are not a main reason these games are that popular. Non outliers reached at most 2 million sales.

In [27]:
# Drop games with more than 2 million sales (Global_Sales>2)
df = df[df["Global_Sales"]<2].reset_index(drop=True)

### Clean up summaries

In [28]:
# What summaries are really short? I'll leave them alone
df = df.reset_index(drop=True)
for i in range(len(df["summary"])):
    if len(df["summary"][i]) < 50:
        print(df["Name"][i], " - ", df["summary"][i])

Art Academy: Home Studio  -  Also known as Art Academy: Atelier.
Catherine  -  Catherine is an action adventure game from Atlus.
Star Fox 64 3D  -  Star Fox 64 is being remade in 3-D for the 3DS.
DJ Star  -  Two main modes include Career and Creation.
Arcania: Gothic 4  -  The Gothic series rolls out more epic RPG action.


In [29]:
# make summaries lower case
df['summary'] = df['summary'].str.lower()

In [30]:
# add leading and trailing spaces in each summary
# this makes sure every word has a space on either side of it
df['summary'] = [" "+df['summary'].iloc[i]+" " for i in range(len(df))]

In [32]:
# turn U.S. into united states
df['summary'] = df['summary'].str.replace(" u.s. ",' united states ')

/var/folders/xx/s138h7zs36sdpnp0qvy3wk2m0000gn/T/ipykernel_64627/1332555576.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['summary'] = df['summary'].str.replace(" u.s. ",' united states ')


In [33]:
# remove special characters from summary - replace all with a space
chars = ['\[', '\]', ',', ';', '.', '!', '?', ':', '\'s', '\'ll', '\'nt', '&', '*', 
         '(', ')', '\"', '“', '”', '®', '–', '-', ' - ', '/', '#', '—']
for i in chars:
    df['summary'] = df['summary'].str.replace(i,' ')
df['summary'] = df['summary'].str.replace('’','')
df['summary'] = df['summary'].str.replace('\'','')

/var/folders/xx/s138h7zs36sdpnp0qvy3wk2m0000gn/T/ipykernel_64627/1235218277.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['summary'] = df['summary'].str.replace(i,' ')
/var/folders/xx/s138h7zs36sdpnp0qvy3wk2m0000gn/T/ipykernel_64627/1235218277.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['summary'] = df['summary'].str.replace(i,' ')


Filler words

In [34]:
# what are the most common words used in all summaries? Check for filler words
from collections import Counter
most_freq = Counter(" ".join(df["summary"]).split()).most_common(100)
#print(most_freq)

In [35]:
# remove fluffer words
words = [' the ', ' and ', ' of ', ' to ', ' a ', ' in ', ' with ', ' your ', ' is ', ' you ', \
         ' as ', ' on ', ' for ', ' from ', ' an ', ' that ', ' by ', ' their ', ' will ', ' or ', \
         ' can ', ' all ', ' are ', ' this ', ' it ', ' into ', ' up ', ' be ', ' has ', \
         ' have ', ' its ', ' them ', ' which ', ' than ', ' but ', ' at ', ' was ', ' what ', ' when ',\
         ' take ', ' even ', ' been ', ' while ', ' who ', ' like ', ' get ', ' where ', ' they ']
for i in words:
    df['summary'] = df['summary'].str.replace(i,' ')

# what are most frequent now?
most_freq = Counter(" ".join(df["summary"]).split()).most_common(100)
#print(most_freq)

### Save df 

In [36]:
# save the cleaned df
df.to_pickle("cleaned_df.pkl")